# Python para Lingüistas

Notebook 8: POS Tagging

Alejandro Ariza

Universitat de Barcelona 2022

En este notebook veremos como realizar etiquetado POS usando NLTK.

Usaremos diferentes etiquetadores y veremos la importancia de seleccionar un buen corpus de entrenamiento.

También aprenderemos más acerca de funciones y métodos.

In [1]:
# Importar nltk
import nltk

In [16]:
# Descargar los paquetes importantes para hoy
nltk.download("brown")
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to C:\Users\infr-usu-
[nltk_data]     fil.aules\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to C:\Users\infr-usu-
[nltk_data]     fil.aules\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

In [3]:
# Importar todos los recursos necesarios para hoy
from nltk import bigrams, trigrams

# Importar numpy
import numpy as np

# Importar codecs
import codecs

# Importar etiquetadores
from nltk import DefaultTagger, AffixTagger, UnigramTagger, BigramTagger, TrigramTagger
from nltk import ClassifierBasedPOSTagger

# Importar el corpus brown
from nltk.corpus import brown

In [4]:
# Funciones avanzadas

# Antes de continuar con el etiquetado POS, observermos el comportamiento de una función muy simple

var_1 = "Boo"
var_2 = "Hoo"
var_3 = "DooDoo"

def my_simple_function(atr_1, atr_2, atr_3="ignórame"):
    
    print("Mi atr_1 es:" + str(atr_1))
    print("Mi atr_2 es:" + str(atr_2))
    print("Mi atr_3 es:" + str(atr_3))



In [5]:
# Veamos lo que hace
my_simple_function(var_1, var_2, var_3)

Mi atr_1 es:Boo
Mi atr_2 es:Hoo
Mi atr_3 es:DooDoo


In [6]:
# ¿Qué pasa si cambiamos el orden?
# Como podéis ver, la función asigna valores a las variables de entrada basándose en su posición
my_simple_function(var_3, var_1, var_2)

Mi atr_1 es:DooDoo
Mi atr_2 es:Boo
Mi atr_3 es:Hoo


In [7]:
# ¿Qué pasa si no añadimos valor para el tercer argumento de entrada?
my_simple_function(var_1, var_2)

Mi atr_1 es:Boo
Mi atr_2 es:Hoo
Mi atr_3 es:ignórame


In [8]:
# ¿Qué pasa si no añadimos valor para el segundo argumento?
my_simple_function(var_1)

TypeError: my_simple_function() missing 1 required positional argument: 'atr_2'

In [9]:
# ¿Qué pasa si le decimos explícitamente qué es cada cosa?
my_simple_function(atr_3=var_1, atr_1=var_2, atr_2=var_3)

Mi atr_1 es:Hoo
Mi atr_2 es:DooDoo
Mi atr_3 es:Boo


### Etiquetado POS en Python

En esta clase, veremos diferentes formas de etiquetar automáticamente un corpus con su POS

Nos centraremos en los etiquetadores basados en n-gramas

In [10]:
# Obtened una lista de categorías del corpus brown
brown.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [11]:
# Conseguid la versión tokenizada y etiquetada de la categoría "news"
brown_twords = brown.tagged_words(categories='news')

# Conseguid la versión que, además, tiene la segmentación de las frases
brown_tsents = brown.tagged_sents(categories='news')

In [12]:
# Imprimid las 5 primeras palabras
print("\nLas primeras 5 palabras en la versión tokenizada y etiquetada son:")
print(brown_twords[:5])


Las primeras 5 palabras en la versión tokenizada y etiquetada son:
[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL')]


In [13]:
# Imprimid las dos primeras frases
print("\nLas primeras 2 frases en la versión con frases segmentadas son:")
print(brown_tsents[:2])


Las primeras 2 frases en la versión con frases segmentadas son:
[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', '

In [14]:
# Conseguid el conjunto de todas las etiquetas que aparecen en el corpus brown
brown_tags = set([tag for (token,tag) in brown_twords])
print("\nEl conjunto de etiquetas en el corpus brown es:")
print(brown_tags)


El conjunto de etiquetas en el corpus brown es:
{'BEM', 'NN$-TL', 'PP$-TL', 'PP$', 'NNS$', 'BEDZ', 'DTI-HL', 'RB+BEZ', 'RBR', 'DTX', 'JJT-HL', 'MD-TL', ',-HL', 'BEZ', 'NNS', 'DT-HL', 'BE', 'JJR', 'FW-IN+NN', 'FW-IN+NN-TL', 'JJS', 'VB-HL', 'RB$', 'AP-HL', 'CS', 'DT', 'EX+BEZ', 'NR-HL', 'NN-TL-HL', ')', 'JJ', 'BEDZ*', '.-HL', 'FW-NNS', 'VB+PPO', 'AP$', 'NPS-HL', 'BEDZ-HL', 'JJ-NC', 'ABN', 'NR$-TL', 'JJS-TL', 'CD-TL', 'NP-HL', 'VBZ-HL', 'QL-TL', 'DO', 'NN', 'FW-IN-TL', 'NNS-HL', 'UH-TL', 'OD-TL', 'NPS', 'NNS-TL-HL', 'WDT', 'NN$-HL', 'BER-TL', 'TO-HL', 'PPSS+HVD', 'PP$$', 'NR', 'PPO', 'NNS-TL', 'PPS+BEZ-HL', 'BED*', 'WP$', 'NP+BEZ', 'RB-TL', 'NPS$', 'VB-TL', 'NP', 'FW-NN', '*', 'CS-HL', 'VBZ', ',', 'TO-TL', 'RP', 'PPSS+HV', 'OD', 'NNS$-TL', '``', 'AT-TL', 'FW-JJ', 'ABX', "''", 'WPS', 'FW-AT-HL', 'FW-AT-TL', 'RB-HL', 'PN+HVZ', 'MD', 'DO-HL', 'HVD', 'JJR-TL', 'AT-HL', 'DOD', "'", 'BE-HL', 'FW-CC', 'WQL', 'IN-TL', 'PN$', 'HV', 'FW-VB-NC', 'CC', 'FW-AT', 'AP-TL', 'PPSS-HL', 'VBD-HL', 'MD*-HL'

In [17]:
# Conseguid el conjunto de todas las etiquetas que existen en la versión "universal"
brown_utwords = brown.tagged_words(categories='news',tagset='universal')
universal_tags = set([tag for (token,tag) in brown_utwords])
print("\nEl conjunto de etiquetas universales es:")
print(universal_tags)


El conjunto de etiquetas universales es:
{'NOUN', 'ADP', '.', 'X', 'DET', 'PRT', 'NUM', 'PRON', 'ADV', 'CONJ', 'VERB', 'ADJ'}


Entrenaremos y evaluaremos diferentes etiquetadores:

- default
- affix
- unigram
- bigram
- trigram

In [18]:
# Etiquetador por defecto

# Obtened la versión segmentada y tokenizada de "news"
# Esta es la versión no etiquetada que vamos a etiquetar
brown_sents = brown.sents(categories='news')

# Obtened una lista de todas las etiquetas en el corpus
tags = [tag for (word, tag) in brown.tagged_words(categories='news')]

# Obtened la etiqueta más frecuente en el corpus
most_frequent_tag = nltk.FreqDist(tags).max()

# Imprimid la etiqueta más frecuente:
print(most_frequent_tag)

NN


In [19]:
# Configurad un etiquetador por defecto
# El etiquetador por defecto asigna la misma etiqueta por defecto a todos los tokens en el corpus
# Lo configuraremos para que anote la etiqueta más frecuente
default_tagger = nltk.DefaultTagger(most_frequent_tag)

my_sent = "the quick brown fox jumped over the lazy dog".split()
print("La frase etiquetada con el etiquetador por defecto:")
print(default_tagger.tag(my_sent))

La frase etiquetada con el etiquetador por defecto:
[('the', 'NN'), ('quick', 'NN'), ('brown', 'NN'), ('fox', 'NN'), ('jumped', 'NN'), ('over', 'NN'), ('the', 'NN'), ('lazy', 'NN'), ('dog', 'NN')]


In [20]:
# Evaluad el etiquetador por defecto en el corpus:
print("La precisión del etiquetador por defecto en el corpus brown es:")
print(round(default_tagger.evaluate(brown_tsents),2))


La precisión del etiquetador por defecto en el corpus brown es:
0.13


In [21]:
# Para el resto de etiquetadores, dividiremos el corpus en train y test:
test_corpus = brown_tsents[:1000]
train_corpus = brown_tsents[1000:]


In [22]:
# Entrena el etiquetador affix
affix_tagger = AffixTagger(train_corpus)

# Etiqueta el corpus
affix_sents = affix_tagger.tag_sents(brown_sents)

# Imprime la primera frase y la precisión
print("\nLa primera frase etiquetada con el etiquetador affix es:")
print(affix_sents[0])
print("\nLa precisión del etiquetador affix en el corpus es:")
print(round(affix_tagger.evaluate(test_corpus),2))


La primera frase etiquetada con el etiquetador affix es:
[('The', None), ('Fulton', 'NP'), ('County', 'NN'), ('Grand', 'NP'), ('Jury', None), ('said', None), ('Friday', 'NR'), ('an', None), ('investigation', 'NN'), ('of', None), ("Atlanta's", 'NP$'), ('recent', 'NN'), ('primary', 'JJ'), ('election', 'NN'), ('produced', 'VBN'), ('``', None), ('no', None), ('evidence', 'NN'), ("''", None), ('that', None), ('any', None), ('irregularities', 'NNS'), ('took', None), ('place', 'NN'), ('.', None)]

La precisión del etiquetador affix en el corpus es:
0.26


In [23]:
# Entrenamiento
unigram_tagger = UnigramTagger(train_corpus) 

# Etiquetado
uni_sents = unigram_tagger.tag_sents(brown_sents)

# Visualización de resultados
print("\nLa primera frase etiquetada con el etiquetador basado en unigramas es:")
print(uni_sents[0])
print("\nLa precisión del etiquetador basado en unigramas en el corpus es:")
print(round(unigram_tagger.evaluate(test_corpus),2))


La primera frase etiquetada con el etiquetador basado en unigramas es:
[('The', 'AT'), ('Fulton', None), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'JJ'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', None), ('took', 'VBD'), ('place', 'NN'), ('.', '.')]

La precisión del etiquetador basado en unigramas en el corpus es:
0.83


In [24]:
# Entrenamiento
bigram_tagger = BigramTagger(train_corpus)

# Etiquetado
bi_sents = bigram_tagger.tag_sents(brown_sents)

# Visualización de resultados
print("\nLa primera frase etiquetada con el etiquetador basado en bigramas es:")
print(bi_sents[0])
print("\nLa precisión del etiquetador basado en bigramas en el corpus es:")
print(round(bigram_tagger.evaluate(test_corpus),2))


La primera frase etiquetada con el etiquetador basado en bigramas es:
[('The', 'AT'), ('Fulton', None), ('County', None), ('Grand', None), ('Jury', None), ('said', None), ('Friday', None), ('an', None), ('investigation', None), ('of', None), ("Atlanta's", None), ('recent', None), ('primary', None), ('election', None), ('produced', None), ('``', None), ('no', None), ('evidence', None), ("''", None), ('that', None), ('any', None), ('irregularities', None), ('took', None), ('place', None), ('.', None)]

La precisión del etiquetador basado en bigramas en el corpus es:
0.1


In [25]:
# Entrenamiento
trigram_tagger = TrigramTagger(train_corpus)

# Etiquetado
tri_sents = trigram_tagger.tag_sents(brown_sents)

tri_sents = trigram_tagger.tag_sents(brown_sents)

# Visualización de resultados
print("\nLa primera frase etiquetada con el etiquetador basado en trigramas es:")
print(tri_sents[0])
print("\nLa precisión del etiquetador basado en trigramas en el corpus es:")
print(round(trigram_tagger.evaluate(test_corpus),2))


La primera frase etiquetada con el etiquetador basado en trigramas es:
[('The', 'AT'), ('Fulton', None), ('County', None), ('Grand', None), ('Jury', None), ('said', None), ('Friday', None), ('an', None), ('investigation', None), ('of', None), ("Atlanta's", None), ('recent', None), ('primary', None), ('election', None), ('produced', None), ('``', None), ('no', None), ('evidence', None), ("''", None), ('that', None), ('any', None), ('irregularities', None), ('took', None), ('place', None), ('.', None)]

La precisión del etiquetador basado en trigramas en el corpus es:
0.06


In [26]:
# Entrenamiento
bigram_tagger_backoff = BigramTagger(train_corpus,backoff=unigram_tagger)

# Etiquetado
bi_sents_bo = bigram_tagger_backoff.tag_sents(brown_sents)


# Visualización de resultados
print("\nLa primera frase etiquetada con el etiquetador basado en bigramas con backoff es:")
print(bi_sents_bo[0])
print("\nLa precisión del etiquetador basado en bigramas con backoff en el corpus es:")
print(round(bigram_tagger_backoff.evaluate(test_corpus),2))


La primera frase etiquetada con el etiquetador basado en bigramas con backoff es:
[('The', 'AT'), ('Fulton', None), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'JJ'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'WPS'), ('any', 'DTI'), ('irregularities', None), ('took', 'VBD'), ('place', 'NN'), ('.', '.')]

La precisión del etiquetador basado en bigramas con backoff en el corpus es:
0.83


In [30]:
# Tarea 1
# Entrenar y evaluar en el mismo corpus en vez de usar particiones separadas
# ¿Cuál es el resultado?

print(round(bigram_tagger_backoff.evaluate(train_corpus),2))

0.97


In [31]:
# Tarea 2
# Crear una secuencia de etiquetadores a modo backoff:
# trigram -> bigram -> unigram -> affix -> default
# Evaluar el etiquetador resultante
default_tagger = nltk.DefaultTagger(most_frequent_tag)
affix_tagger = AffixTagger(train_corpus, backoff=default_tagger)
unigram_tagger = UnigramTagger(train_corpus, backoff=affix_tagger)
bigram_tagger = BigramTagger(train_corpus,backoff=unigram_tagger)
trigram_tagger = TrigramTagger(train_corpus, backoff=bigram_tagger)

tri_sents_bo = trigram_tagger.tag_sents(brown_sents)


# Visualización de resultados
print("\nLa primera frase etiquetada con el etiquetador basado en bigramas con backoff es:")
print(tri_sents_bo[0])
print("\nLa precisión del etiquetador basado en bigramas con backoff en el corpus es:")
print(round(trigram_tagger.evaluate(test_corpus),2))


La primera frase etiquetada con el etiquetador basado en bigramas con backoff es:
[('The', 'AT'), ('Fulton', 'NP'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'JJ'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'WPS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')]

La precisión del etiquetador basado en bigramas con backoff en el corpus es:
0.88


In [33]:
# Tarea 3
# Intentar cross-domain tagging:
# Entrenar usando el corpus "news" y evaluar en "science_fiction"
# Comparar los resultados con un corpus entrenado y evaluado en el mismo dominio (science_fiction)
news_sents = brown.tagged_sents(categories='news')
science_fiction_sents = brown.tagged_sents(categories='science_fiction')

# Entrena el etiquetador affix
affix_tagger = AffixTagger(news_sents)

# Etiqueta el corpus
affix_sents = affix_tagger.tag_sents(news_sents)

# Imprime la primera frase y la precisión
print("\nLa primera frase etiquetada con el etiquetador affix es:")
print(affix_sents[0])
print("\nLa precisión del etiquetador affix en el corpus es:")
print(round(affix_tagger.evaluate(science_fiction_sents),2))


La primera frase etiquetada con el etiquetador affix es:
[(('The', 'AT'), None), (('Fulton', 'NP-TL'), None), (('County', 'NN-TL'), None), (('Grand', 'JJ-TL'), None), (('Jury', 'NN-TL'), None), (('said', 'VBD'), None), (('Friday', 'NR'), None), (('an', 'AT'), None), (('investigation', 'NN'), None), (('of', 'IN'), None), (("Atlanta's", 'NP$'), None), (('recent', 'JJ'), None), (('primary', 'NN'), None), (('election', 'NN'), None), (('produced', 'VBD'), None), (('``', '``'), None), (('no', 'AT'), None), (('evidence', 'NN'), None), (("''", "''"), None), (('that', 'CS'), None), (('any', 'DTI'), None), (('irregularities', 'NNS'), None), (('took', 'VBD'), None), (('place', 'NN'), None), (('.', '.'), None)]

La precisión del etiquetador affix en el corpus es:
0.19


In [35]:
# Tarea 4
# Importante del conjunto de etiquetas
# Re-ejecutar las tareas 2 y 3 usando el etiquetado universal del corpus
# Comparar las diferencias entre el etiquetado "universal" y el etiquetado completo
news_sents = brown.tagged_sents(categories='news', tagset="universal")
science_fiction_sents = brown.tagged_sents(categories='science_fiction', tagset='universal')

# Entrena el etiquetador affix
affix_tagger = AffixTagger(news_sents)

# Etiqueta el corpus
affix_sents = affix_tagger.tag_sents(news_sents)

# Imprime la primera frase y la precisión
print("\nLa primera frase etiquetada con el etiquetador affix es:")
print(affix_sents[0])
print("\nLa precisión del etiquetador affix en el corpus es:")
print(round(affix_tagger.evaluate(science_fiction_sents),2))


La primera frase etiquetada con el etiquetador affix es:
[(('The', 'DET'), None), (('Fulton', 'NOUN'), None), (('County', 'NOUN'), None), (('Grand', 'ADJ'), None), (('Jury', 'NOUN'), None), (('said', 'VERB'), None), (('Friday', 'NOUN'), None), (('an', 'DET'), None), (('investigation', 'NOUN'), None), (('of', 'ADP'), None), (("Atlanta's", 'NOUN'), None), (('recent', 'ADJ'), None), (('primary', 'NOUN'), None), (('election', 'NOUN'), None), (('produced', 'VERB'), None), (('``', '.'), None), (('no', 'DET'), None), (('evidence', 'NOUN'), None), (("''", '.'), None), (('that', 'ADP'), None), (('any', 'DET'), None), (('irregularities', 'NOUN'), None), (('took', 'VERB'), None), (('place', 'NOUN'), None), (('.', '.'), None)]

La precisión del etiquetador affix en el corpus es:
0.24
